In [1]:
#################### LOADING ########################

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import csv
import cesium
from cesium import featurize


# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore", category=RuntimeWarning)
simplefilter(action='ignore', category=FutureWarning)

# Evaluation metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

%matplotlib inline

# variavel que evita NaN nos resultados
epslon = 0.00005
def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_pred - y_true)/np.sum(y_true))
  else:
   return 100*(np.sum(y_pred - y_true)/(np.sum(y_true)+ epslon))  

def pocid(y_true, y_pred):
  n = len(y_true)
  D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
  POCID = 100 * np.sum(D) / (n-1)
  return POCID

#função para normalização
def znorm(x):
  if np.std(x)!=0:
    x_znorm = (x - np.mean(x)) / np.std(x)
  else:
    x_znorm = (x - np.mean(x)) / (np.std(x) + epslon) 
   
  return x_znorm

#função para desnormatização
def znorm_reverse(x, mean_x, std_x):
  x_denormalized = (np.array(x) * std_x) + mean_x
  return x_denormalized

def get_stats_norm(series, horizon, window):
  last_subsequence = series[-(horizon+window):-horizon].values
  last_mean = np.mean(last_subsequence)
  last_std = np.std(last_subsequence)
  return last_mean, last_std

# Em geral, considera-se um tamanho de janela capaz de capturar um ciclo dos dados
# Por exemplo, 12 observações no caso dos dados com frequência mensal
def rolling_window(series, window):
  data = []
  for i in range(len(series)-window):
    example = znorm(np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df

def rolling_window_real(series, window):
  data = []
  for i in range(len(series)-window):
    example = (np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df

# Para predição de vendas por UF (mensal), será considerado horizon = 12
# Para predição de vendas por município (anual), será considerado horizon = 1
def train_test_split(data, horizon):
  X = data.iloc[:,:-1] # features
  y = data.iloc[:,-1] # target

  X_train = X[:-horizon] # features train
  X_test =  X[-horizon:] # features test

  y_train = y[:-horizon] # target train
  y_test = y[-horizon:] # target test
  return X_train, X_test, y_train, y_test

def recursive_multistep_forecasting(X_test, model, horizon):
  # example é composto pelas últimas observações vistas
  # na prática, é o pbeprimeiro exemplo do conjunto de teste
  example = X_test.iloc[0].values.reshape(1,-1)

  preds = []
  for i in range(horizon):
    pred = model.predict(example)[0]
    preds.append(pred)

    # Descartar o valor da primeira posição do vetor de características
    example = example[:,1:]

    # Adicionar o valor predito na última posição do vetor de características
    example = np.append(example, pred)
    example = example.reshape(1,-1)
  return preds

#features general > 27 erro > 44 periodic > 47 = total 117
features_to_use_general = [
    "amplitude",
    "anderson_darling",
    "flux_percentile_ratio_mid20",
    "flux_percentile_ratio_mid35",
    "flux_percentile_ratio_mid50",
    "flux_percentile_ratio_mid65",
    "flux_percentile_ratio_mid80",
    "percent_beyond_1_std",
    "percent_amplitude",
    "percent_beyond_1_std",
    "percent_difference_flux_percentile",
    "period_fast",
    "qso_log_chi2_qsonu",
    "qso_log_chi2nuNULL_chi2nu",
    "shapiro_wilk",
    "stetson_j",
    "stetson_k",
    "weighted_average",
    "maximum",
    "max_slope",
    "median",
    "median_absolute_deviation",
    "percent_close_to_median",
    "minimum",
    "skew",
    "std",
    "weighted_average",
]

features_to_use_cadenceError = [
    "all_times_nhist_numpeaks",
    "all_times_nhist_peak1_bin",
    "all_times_nhist_peak2_bin",
    "all_times_nhist_peak3_bin",
    "all_times_nhist_peak4_bin",
    "all_times_nhist_peak_1_to_2",
    "all_times_nhist_peak_1_to_3",
    "all_times_nhist_peak_1_to_4",
    "all_times_nhist_peak_2_to_3",
    "all_times_nhist_peak_2_to_4",
    "all_times_nhist_peak_3_to_4",
    "all_times_nhist_peak_val",
    "avg_double_to_single_step",
    "avg_err",
    "avgt",
    "cad_probs_1",
    "cad_probs_10",
    "cad_probs_20",
    "cad_probs_30",
    "cad_probs_40",
    "cad_probs_50",
    "cad_probs_100",
    "cad_probs_500",
    "cad_probs_1000",
    "cad_probs_5000",
    "cad_probs_10000",
    "cad_probs_50000",
    "cad_probs_100000",
    "cad_probs_500000",
    "cad_probs_1000000",
    "cad_probs_5000000",
    "cad_probs_10000000",
    "cads_avg",
    "cads_kurtosis",
    "cads_med",
    "cads_skew",
    "cads_std",
    "mean",
    "med_err",
    "n_epochs",
    "std_double_to_single_step",
    "std_err",
    "total_time",
]


features_to_use_lombScargle = [
    "fold2P_slope_10percentile",
    "fold2P_slope_90percentile",
    "freq1_amplitude1",
    "freq1_amplitude2",
    "freq1_amplitude3",
    "freq1_amplitude4",
    "freq1_freq",
    "freq1_lambda",
    "freq1_rel_phase2",
    "freq1_rel_phase3",
    "freq1_rel_phase4",
    "freq1_signif",
    "freq2_amplitude1",
    "freq2_amplitude2",
    "freq2_amplitude3",
    "freq2_amplitude4",
    "freq2_freq",
    "freq2_rel_phase2",
    "freq2_rel_phase3",
    "freq2_rel_phase4",
    "freq3_amplitude1",
    "freq3_amplitude2",
    "freq3_amplitude3",
    "freq3_amplitude4",
    "freq3_freq",
    "freq3_rel_phase2",
    "freq3_rel_phase3",
    "freq3_rel_phase4",
    "freq_amplitude_ratio_21",
    "freq_amplitude_ratio_31",
    "freq_frequency_ratio_21",
    "freq_frequency_ratio_31",
    "freq_model_max_delta_mags",
    "freq_model_min_delta_mags",
    "freq_model_phi1_phi2",
    "freq_n_alias",
    "freq_signif_ratio_21",
    "freq_signif_ratio_31",
    "freq_varrat",
    "freq_y_offset",
    "linear_trend",
    "medperc90_2p_p",
    "p2p_scatter_2praw",
    "p2p_scatter_over_mad",
    "p2p_scatter_pfold_over_mad",
    "p2p_ssqr_diff_over_var",
    "scatter_res_raw",
]

features_to_use_all = [
    "amplitude",
    "anderson_darling",
    "flux_percentile_ratio_mid20",
    "flux_percentile_ratio_mid35",
    "flux_percentile_ratio_mid50",
    "flux_percentile_ratio_mid65",
    "flux_percentile_ratio_mid80",
    "percent_beyond_1_std",
    "percent_amplitude",
    "percent_beyond_1_std",
    "percent_difference_flux_percentile",
    "period_fast",
    "qso_log_chi2_qsonu",
    "qso_log_chi2nuNULL_chi2nu",
    "shapiro_wilk",
    "stetson_j",
    "stetson_k",
    "weighted_average",
    "maximum",
    "max_slope",
    "median",
    "median_absolute_deviation",
    "percent_close_to_median",
    "minimum",
    "skew",
    "std",
    "weighted_average",
    "all_times_nhist_numpeaks",
    "all_times_nhist_peak1_bin",
    "all_times_nhist_peak2_bin",
    "all_times_nhist_peak3_bin",
    "all_times_nhist_peak4_bin",
    "all_times_nhist_peak_1_to_2",
    "all_times_nhist_peak_1_to_3",
    "all_times_nhist_peak_1_to_4",
    "all_times_nhist_peak_2_to_3",
    "all_times_nhist_peak_2_to_4",
    "all_times_nhist_peak_3_to_4",
    "all_times_nhist_peak_val",
    "avg_double_to_single_step",
    "avg_err",
    "avgt",
    "cad_probs_1",
    "cad_probs_10",
    "cad_probs_20",
    "cad_probs_30",
    "cad_probs_40",
    "cad_probs_50",
    "cad_probs_100",
    "cad_probs_500",
    "cad_probs_1000",
    "cad_probs_5000",
    "cad_probs_10000",
    "cad_probs_50000",
    "cad_probs_100000",
    "cad_probs_500000",
    "cad_probs_1000000",
    "cad_probs_5000000",
    "cad_probs_10000000",
    "cads_avg",
    "cads_kurtosis",
    "cads_med",
    "cads_skew",
    "cads_std",
    "mean",
    "med_err",
    "n_epochs",
    "std_double_to_single_step",
    "std_err",
    "total_time",
    "fold2P_slope_10percentile",
    "fold2P_slope_90percentile",
    "freq1_amplitude1",
    "freq1_amplitude2",
    "freq1_amplitude3",
    "freq1_amplitude4",
    "freq1_freq",
    "freq1_lambda",
    "freq1_rel_phase2",
    "freq1_rel_phase3",
    "freq1_rel_phase4",
    "freq1_signif",
    "freq2_amplitude1",
    "freq2_amplitude2",
    "freq2_amplitude3",
    "freq2_amplitude4",
    "freq2_freq",
    "freq2_rel_phase2",
    "freq2_rel_phase3",
    "freq2_rel_phase4",
    "freq3_amplitude1",
    "freq3_amplitude2",
    "freq3_amplitude3",
    "freq3_amplitude4",
    "freq3_freq",
    "freq3_rel_phase2",
    "freq3_rel_phase3",
    "freq3_rel_phase4",
    "freq_amplitude_ratio_21",
    "freq_amplitude_ratio_31",
    "freq_frequency_ratio_21",
    "freq_frequency_ratio_31",
    "freq_model_max_delta_mags",
    "freq_model_min_delta_mags",
    "freq_model_phi1_phi2",
    "freq_n_alias",
    "freq_signif_ratio_21",
    "freq_signif_ratio_31",
    "freq_varrat",
    "freq_y_offset",
    "linear_trend",
    "medperc90_2p_p",
    "p2p_scatter_2praw",
    "p2p_scatter_over_mad",
    "p2p_scatter_pfold_over_mad",
    "p2p_ssqr_diff_over_var",
    "scatter_res_raw",
]


def rolling_window_celsium(series, window):
  data_out3 = []
  for i in range(len(series)-window):
      example = ((series[i:i+window+1]))
      example2 = example.iloc[:-1]
      fset_cesium = featurize.featurize_time_series(times=example2["timestamp"],values=example2["m3"].values, errors=None,features_to_use=features_to_use_all)
      new_elements_values_reshaped = np.squeeze(fset_cesium.values) 
      data_featuredf = pd.DataFrame(new_elements_values_reshaped)
      ###### Norm ############################
      # data_featuredf.replace([np.inf, -np.inf], 0, inplace=True)
      # data_featuredf.drop(data_featuredf.columns[[8, 10]], axis=1, inplace=True)
      # data_featuredf2 = znorm(data_featuredf)
      ##################################
      data_out3.append(data_featuredf.T.values)
  data_out4 = np.squeeze(data_out3) 
  df2 = pd.DataFrame((data_out4))
  return df2


C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
#### LEITURA DOS DADOS ####

def extract_estado(file_name):
    # Split the file name by underscores
    parts = file_name.split('_')
    # Extract the name between underscores
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    # List all files in the folder
    files = os.listdir(folder_path)
    # Iterate through each file
    for file_name in files:
        # Check if it's a CSV file
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            # Open the CSV file and read the data
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                # Assuming the first row contains headers
                headers = next(reader)
                # Extract estado from file name and append to estados list
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados



In [8]:
############## TsCesium AUTO ##############

import pickle

horizon = 12
window = 12
features = 'TsCesium'

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    
    for estado in estados:

        # carregamento do arquivo

        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
        series = df

        outTsCelsium = rolling_window_celsium(series, window) 
        seriesy = series.iloc[12:]
        seriesy2= seriesy.reset_index()

        outTsCelsium = outTsCelsium.fillna(0)
        
        TimeStamp = df['timestamp'].tail(398) 
        TimeStamp.reset_index(drop=True, inplace=True) 
 
        outTsCelsium.insert(0, 'timestamp', TimeStamp) ## add

        folder_name = f'TsCesium'
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        
        outTsCelsium.to_csv(f'{folder_name}/FEAT_TsCesium_{product}_{window}_{estado}.csv', index=False)


KeyboardInterrupt: 